In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
vl_path = os.path.join("..", "data", "transformed","vention_transformed_vl_since_2022.csv")
uam_path = os.path.join("..", "data", "transformed","vention_transformed_uam_ga_client_ids.csv")

df_vl = pd.read_csv(vl_path) 
df_uam = pd.read_csv(uam_path) 

C:\Users\marie-eve.malette_ve\AppData\Local\Temp\ipykernel_18824\2020465127.py:4: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vl = pd.read_csv(vl_path)


In [3]:
# format dates in vl 
df_vl = df_vl[['user_id', 'created_at', 'ga_client_id', 'view_data']]
df_vl['created_at'] = pd.to_datetime(df_vl['created_at'], format='%Y-%m-%d %H:%M:%S')

# format and cleanup
df_vl = df_vl[~df_vl['user_id'].astype(str).str.match(r'\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}$')]
df_vl['user_id'] = [str(i) if str(i) == 'nan' else str(int(i)) for i in df_vl['user_id']]
df_vl['ga_client_id'] = df_vl['ga_client_id'].astype(str)

# remove from view_ogs rows where both ga and user ids are nan
df_vl = df_vl.loc[(df_vl['ga_client_id'] != 'nan')]

In [4]:
# fromat and cleanup UAM
df_uam['user_id'] = df_uam['user_id'].astype(str) 
df_uam = df_uam[['user_id','ga_client_id']]

In [5]:
# retrieve missing user_id in vl
df_vl_uam = pd.merge(df_vl, df_uam, left_on="ga_client_id", right_on="ga_client_id", how='left')

# remove still null user_id
df_vl_uam = df_vl_uam.loc[~df_vl_uam['user_id_y'].isnull()]

# drop unecessary columns, rename columns and reset indices
df_vl_uam.drop(['user_id_x', 'ga_client_id'], axis=1, inplace=True)
df_vl_uam.rename(columns={'user_id_y': 'user_id_aug'}, inplace=True)
df_vl_uam.reset_index(drop=True, inplace=True)

In [6]:
def get_url(view_data):
    view_data = eval(view_data)
    if view_data != "":
        url = view_data['url']
    else:
        url = None
        
    return url

def parse_url(url_suffix):
    # print(url_suffix)
    url_parts = url_suffix.split('/')

    try:
        url_suffix = url_parts[0]
    except IndexError: 
        url_suffix = 'vention.io'
        
    return url_suffix


def parse_url2(url_suffix2):
    if url_suffix2.startswith(('de?', '?')):
        url_suffix3 = 'probable-source-google-campaign'
    else: 
        url_parts = url_suffix2.split('?')
        url_suffix3 = url_parts[0]
        
    return url_suffix3

In [7]:
# extract the url from view_data
df_vl_uam['url'] = df_vl_uam['view_data'].apply(lambda x: get_url(x))

# extract the suffix (what comes after vention.io/)
df_vl_uam['url_suffix'] = ''
df_vl_uam['url_suffix'] = df_vl_uam['url'].str.replace('https://vention.io/', '')
df_vl_uam['url_suffix'] = df_vl_uam['url_suffix'].astype(str)

# parsing the url_suffix even further by keeping only what comes before the first '/'
df_vl_uam['url_suffix2'] = None
df_vl_uam['url_suffix2'] = df_vl_uam['url_suffix'].apply(lambda x: parse_url(x))
# after this, the number of categories is still very high: above 113K

C:\Users\marie-eve.malette_ve\AppData\Local\Temp\ipykernel_18824\3194990985.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_vl_uam['url_suffix'] = df_vl_uam['url'].str.replace('https://vention.io/', '')


In [ ]:
# parsing even more to reduce the number of categorie to 6-700 
df_vl_uam['url_suffix3'] = None
df_vl_uam['url_suffix3'] = df_vl_uam['url_suffix2'].apply(lambda x: parse_url2(x))
url_suffix3_vc = df_vl_uam['url_suffix3'].value_counts()

# replace url that are infrequent with the word infrequent
url_suffix3_vc = pd.DataFrame({'url_suffix': url_suffix3_vc.index, 'count': list(url_suffix3_vc)})
url_renames = list(url_suffix3_vc.loc[url_suffix3_vc['count'] < 10, 'url_suffix'].unique())
df_vl_uam.loc[df_vl_uam['url_suffix3'].isin(url_renames), 'url_suffix3'] = 'infrequent'

# replacing empty strings with vention.io, url with url_suffix3
df_vl_uam.loc[df_vl_uam['url_suffix3'] == '', 'url_suffix3'] = 'vention.io'
df_vl_uam['url'] = df_vl_uam['url_suffix3']

In [ ]:
# cleanup and renaming
df_vl_uam_final = df_vl_uam.drop(['view_data',  'url_suffix', 'url_suffix2', 'url_suffix3'], axis=1)
df_vl_uam_final.rename(columns={'created_at': 'vl_created_at'}, inplace=True)


In [ ]:
df_vl_uam_final.to_csv('tmp/df_view_logs_uam_final.csv')